In [2]:
# def warn(*args, **kwargs):
#     pass
# import warnings
# warnings.warn = warn

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import sqlite3
# import seaborn as sns

# ### Setting Up DB Connection ###
# db = 'database.sqlite'
# conn = sqlite3.connect(db)

# matches = pd.read_sql("""SELECT c.name, l.name, m.season, 
#             m.home_team_api_id, m.away_team_api_id, m.home_team_goal, m.away_team_goal
#             FROM Match as m
#             INNER JOIN Country as c
#             ON m.country_id = c.id
#             INNER JOIN League as l
#             ON m.league_id = l.id
#             WHERE c.name IN ('England', 'Germany', 'Italy', 'Spain')""", conn)

# teams = pd.read_sql("""SELECT team_api_id, team_long_name
#                         FROM Team""", conn)

# for idx, row in matches.iterrows():
#     for idx2, row2 in teams.iterrows():
#         if row['home_team_api_id'] == row2['team_api_id']:
#             matches.loc[idx, 'home_team_api_id'] = row2['team_long_name']
#         elif row['away_team_api_id'] == row2['team_api_id']:
#             matches.loc[idx, 'away_team_api_id'] = row2['team_long_name']

matches.columns = ['country', 'league', 'season', 'home_team', 'away_team', 'home_team_goal', 'away_team_goal']
matches.head(3)

,country,league,season,home_team,away_team,home_team_goal,away_team_goal
0,England,England Premier League,2008/2009,Manchester United,Newcastle United,1,1
1,England,England Premier League,2008/2009,Arsenal,West Bromwich Albion,1,0
2,England,England Premier League,2008/2009,Sunderland,Liverpool,0,1


In [47]:
total_team_dfs = []
for league in list(matches.league.unique()):
    for season in list(matches.season.unique()):
        league_season = matches.loc[(matches['league'] == league) & (matches['season'] == season)]
        for team in list(league_season.home_team.unique()):
            team_df = league_season.loc[(league_season['home_team'] == team) | (league_season['away_team'] == team)]
            team_df['home_wins'] = 0
            team_df['away_wins'] = 0
            team_df['ties'] = 0
            team_df['home_losses'] = 0
            team_df['away_losses'] = 0
            team_df['goals_scored_home'] = 0
            team_df['goals_scored_away'] = 0
            team_df['goals_conceded_home'] = 0
            team_df['goals_conceded_away'] = 0
            team_df['team'] = team
            team_df['home_points'] = 0
            team_df['away_points'] = 0
            for idx, row in team_df.iterrows():
                if row['home_team'] == team:
                    team_df.loc[idx, 'goals_scored_home'] = team_df.loc[idx, 'home_team_goal']
                    team_df.loc[idx, 'goals_conceded_home'] = team_df.loc[idx, 'away_team_goal']
                elif row['away_team'] == team:
                    team_df.loc[idx, 'goals_scored_away'] = team_df.loc[idx, 'away_team_goal']
                    team_df.loc[idx, 'goals_conceded_away'] = team_df.loc[idx, 'home_team_goal']
                if row['home_team'] == team and row['home_team_goal'] > row['away_team_goal']:
                    team_df.loc[idx, 'home_wins'] = 1
                    team_df.loc[idx, 'home_points'] = 3
                elif row['away_team'] == team and row['home_team_goal'] < row['away_team_goal']:
                    team_df.loc[idx, 'away_wins'] = 1
                    team_df.loc[idx, 'away_points'] = 3
                elif row['home_team'] == team and row['home_team_goal'] == row['away_team_goal']:
                    team_df.loc[idx, 'ties'] = 1
                    team_df.loc[idx, 'home_points'] = 1
                elif row['away_team'] == team and row['home_team_goal'] == row['away_team_goal']:
                    team_df.loc[idx, 'ties'] = 1
                    team_df.loc[idx, 'away_points'] = 1
                elif row['home_team'] == team and row['home_team_goal'] < row['away_team_goal']:
                    team_df.loc[idx, 'home_losses'] = 1
                    team_df.loc[idx, 'home_points'] = 0
                elif row['away_team'] == team and row['home_team_goal'] > row['away_team_goal']:
                    team_df.loc[idx, 'away_losses'] = 1
                    team_df.loc[idx, 'away_points'] = 0
            total_team_dfs.append(team_df)
total_matches = pd.concat(total_team_dfs)
            

   

                
                

In [50]:
total_matches.to_csv('total_matches.csv', index=False)